This notebook contains code to download the CLIP-ViT-B-32 model from Hugging Face and deploy it to a SageMaker serverless endpoint.

### Download  CLIP-ViT-B-32 model

In [1]:
from huggingface_hub import snapshot_download

snapshot_download(repo_id="sentence-transformers/clip-ViT-B-32", local_dir="./CLIP-ViT-B-32")


C:\Users\tochi\anaconda3\envs\upwork-dev\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 11 files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 14.44it/s]


'C:\\Users\\tochi\\OneDrive\\Documents\\Upwork_Job\\clip-embedding-api\\CLIP-ViT-B-32'

### Package and Upload CLIP-ViT-B-32 model to s3 bucket

In [2]:
%cd  CLIP-ViT-B-32
!tar zcvf model.tar.gz *

C:\Users\tochi\OneDrive\Documents\Upwork_Job\clip-embedding-api\CLIP-ViT-B-32


a .cache
a .gitattributes
a 0_CLIPModel
a config_sentence_transformers.json
a model.tar.gz
a modules.json
a README.md
a 0_CLIPModel/config.json
a 0_CLIPModel/merges.txt
a 0_CLIPModel/preprocessor_config.json
a 0_CLIPModel/pytorch_model.bin
a 0_CLIPModel/special_tokens_map.json
a 0_CLIPModel/tokenizer_config.json
a 0_CLIPModel/vocab.json
a .cache/huggingface
a .cache/huggingface/.gitignore
a .cache/huggingface/download
a .cache/huggingface/download/.gitattributes.metadata
a .cache/huggingface/download/0_CLIPModel
a .cache/huggingface/download/config_sentence_transformers.json.metadata
a .cache/huggingface/download/modules.json.metadata
a .cache/huggingface/download/README.md.metadata
a .cache/huggingface/download/0_CLIPModel/config.json.metadata
a .cache/huggingface/download/0_CLIPModel/merges.txt.metadata
a .cache/huggingface/download/0_CLIPModel/preprocessor_config.json.metadata
a .cache/huggingface/download/0_CLIPModel/pytorch_model.bin.metadata
a .cache/huggingface/download/0_CLIPMo

In [3]:
import sagemaker
import boto3

sess = sagemaker.Session()
account = boto3.client("sts").get_caller_identity().get("Account")
role = f"arn:aws:iam::{account}:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole"

C:\Users\tochi\anaconda3\envs\upwork-dev\lib\site-packages\pydantic\_internal\_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[05/24/25 04:09:00] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=287087;file://C:\Users\tochi\anaconda3\envs\upwork-dev\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=840475;file://C:\Users\tochi\anaconda3\envs\upwork-dev\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\tochi\AppData\Local\sagemaker\sagemaker\config.yaml


[05/24/25 04:09:01] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=928116;file://C:\Users\tochi\anaconda3\envs\upwork-dev\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=937716;file://C:\Users\tochi\anaconda3\envs\upwork-dev\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

[05/24/25 04:09:02] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=670058;file://C:\Users\tochi\anaconda3\envs\upwork-dev\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=901644;file://C:\Users\tochi\anaconda3\envs\upwork-dev\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

In [4]:
bucket = "huggingface-clip-models"
model_artifact = sagemaker.Session().upload_data("model.tar.gz", bucket=bucket, key_prefix=" CLIP-ViT-B-32")
print("S3 Model Path:", model_artifact)

S3 Model Path: s3://huggingface-clip-models/ CLIP-ViT-B-32/model.tar.gz


### Create a Sagemaker Huggingface Model

In [11]:
from sagemaker.huggingface.model import HuggingFaceModel

# Create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data = model_artifact,
   role=role,                                          
   transformers_version="4.26",                          
   pytorch_version="1.13",                              
   py_version='py39'                               
)

[05/22/25 23:35:43] INFO     Found credentials from IAM Role:                                   ]8;id=383431;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=584607;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [15]:
from sagemaker.serverless import ServerlessInferenceConfig

# Define serverless configuration
serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=3072,
    max_concurrency=5       
)

In [16]:
predictor = huggingface_model.deploy(
    serverless_inference_config=serverless_config,
    endpoint_name="clip-vit-b32"
)

[05/22/25 23:39:03] INFO     Defaulting to CPU type when using serverless inference               ]8;id=263125;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=327409;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/image_uris.py#538\538]8;;\

                    INFO     Creating model with name:                                              ]8;id=258602;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=831966;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\
                             huggingface-pytorch-inference-2025-05-22-23-39-03-761                                 

[05/22/25 23:39:04] INFO     Creating endpoint-config with name clip-vit-b32                        ]8;id=237329;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=777830;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#6019\6019]8;;\

                    INFO     Creating endpoint with name clip-vit-b32                               ]8;id=486908;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=960508;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4841\4841]8;;\

----!

### Test Deployed Endpoint

In [18]:
# Prepare base64 encoded image
import base64

# Load an image (make sure to use an RGB image like PNG or JPG)
with open("aws_login.png", "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode('utf-8')

# Define payload with base64 image
data = {
  "inputs": {
    "image": encoded_image
  }
}

# Send prediction request to endpoint
response = predictor.predict(data)
print(len(response))
print(response)

512
[0.06883295625448227, 0.05073244869709015, -0.19384917616844177, 0.0483168363571167, 0.4674297571182251, -0.14501774311065674, -0.0009664297103881836, -0.0013284459710121155, 0.10332383215427399, 0.12118306756019592, 0.23240751028060913, 0.12693674862384796, -0.3725017011165619, 0.0022184550762176514, -0.3040201663970947, 0.49554067850112915, 0.09968911856412888, 0.14243429899215698, 0.2675679922103882, 0.0017742961645126343, 0.8995280861854553, 0.33221715688705444, -0.029950633645057678, -0.09795137494802475, 0.03955888748168945, 0.23206937313079834, -0.23208549618721008, 0.3274761438369751, -0.008250698447227478, 0.23895308375358582, -0.04064276069402695, 0.4726744294166565, -0.28956279158592224, 0.36877092719078064, 1.0801054239273071, -0.25806254148483276, -0.10077999532222748, -0.05344244837760925, -0.08659671247005463, -2.1564371585845947, -0.27350184321403503, -0.39885473251342773, 0.2571973204612732, -0.061588287353515625, 0.19745716452598572, -1.0242700576782227, -0.417916